In [ ]:
import boto3
from datetime import datetime
import os

In [ ]:
s3 = boto3.client("s3")
dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table(os.environ["TABLE_NAME"])

In [ ]:
def handler(event, context):
    for record in event["Records"]:
        event_name = record["eventName"]
        src_bucket = record["s3"]["bucket"]["name"]
        object_key = record["s3"]["object"]["key"]

        if "ObjectCreated" in event_name:
            handle_put_event(object_key, src_bucket)
        elif "ObjectRemoved" in event_name:
            handle_delete_event(object_key)

def handle_put_event(object_key, src_bucket):
    dst_bucket = os.environ["BUCKET_DST"]
    timestamp = int(datetime.now().timestamp())
    copy_key = f"{object_key}-{timestamp}"

    # Copy object to destination bucket
    s3.copy_object(
        Bucket=dst_bucket,
        CopySource={"Bucket": src_bucket, "Key": object_key},
        Key=copy_key
    )

    # Update DynamoDB table with new version
    table.put_item(
        Item={
            "ObjectKey": object_key,
            "Version": timestamp,
            "CopyObjectKey": copy_key,
            "Status": "active"
        }
    )

def handle_delete_event(object_key):
    # Update DynamoDB to mark the object as disowned
    items = table.query(
        KeyConditionExpression=boto3.dynamodb.conditions.Key("ObjectKey").eq(object_key)
    )["Items"]

    for item in items:
        table.update_item(
            Key={"ObjectKey": object_key, "Version": item["Version"]},
            UpdateExpression="SET #s = :s, DisownedTimestamp = :ts",
            ExpressionAttributeNames={"#s": "Status"},
            ExpressionAttributeValues={":s": "disowned", ":ts": int(datetime.now().timestamp())}
        )